### 環境建置

In [2]:
# TEST necessary for when working with external scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneGroupOut, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report

### 資料讀取

In [4]:
df = pd.read_csv("../WESAD/lab2_compressed_data_with_features.csv")

# 分開特徵和標籤

X = df.drop(columns=['Label', 'Subject'])  # 移除標籤 (y) 和 Subject
y = df['Label'] 



# 切分訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


FileNotFoundError: [Errno 2] No such file or directory: '../Lab1/WESAD/lab2_compressed_data_with_features.csv'

### 模型

決策樹

In [7]:
# 初始化並訓練決策樹
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
model_name = "DecisionTree"

KNN

In [52]:

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
model_name = "KNN"


隨機森林

In [54]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
model_name = "RandomForest"



AdaBoost 決策樹

In [56]:

model = AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42)
model.fit(X_train, y_train)
model_name = "Adaboost Decision Tree"

/Users/sean/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


線性判別分析 (LDA)


In [58]:
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)
model_name = "LDA"

### 生成結果


In [8]:
# 預測測試集
print(model_name)
y_pred = model.predict(X_test)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred)
print(f"模型準確率: {accuracy:.4f}")

# 輸出分類報告
print("分類報告：")
print(classification_report(y_test, y_pred, digits=4))


DecisionTree
模型準確率: 0.9132
分類報告：
              precision    recall  f1-score   support

           0     0.9218    0.9219    0.9218   5149190
           1     0.9758    0.9753    0.9756   2297210
           2     0.9182    0.9187    0.9184   1293681
           3     0.8286    0.8269    0.8278    729819
           4     0.8620    0.8612    0.8616   1550742
           5     0.7323    0.7382    0.7353    102547
           6     0.8135    0.8186    0.8161    102005
           7     0.6898    0.6969    0.6933    107526

    accuracy                         0.9132  11332720
   macro avg     0.8427    0.8447    0.8437  11332720
weighted avg     0.9132    0.9132    0.9132  11332720



In [9]:
logo = LeaveOneGroupOut()
y_pred = cross_val_predict(model, X, y, cv=logo, groups=df['Subject'])
accuracy = accuracy_score(y, y_pred)
print(f"模型準確率: {accuracy:.4f}")
report = classification_report(y, y_pred, digits=4)
print("分類報告：")
print(report)